# Machine Learning Prediction 

In [466]:
import pandas as pd
import scipy as sc
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression 
from sklearn.linear_model import Lasso # Lasso algorithm
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import BayesianRidge # Bayesian algorithm
from sklearn.linear_model import ElasticNet # ElasticNet algorithm

from sklearn.metrics import r2_score

#!pip install plotly
#!pip install -U kaleido

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import seaborn as sns
import matplotlib as mpl
from matplotlib import pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")


## Importing the data

In [467]:
forsale_df=pd.read_csv('RealEstateNewYork_Clean.csv',sep=',',low_memory=False)
forsale_df.drop(forsale_df.filter(regex="Unname"),axis=1, inplace=True)

In [468]:
forsale_df

,price,beds,baths,garage,stories,house_type,lot_sqft,sqft,year_built,address,state,city,county
0,139900.0,3.0,2.0,0.0,2.0,single_family,12632.0,1190.0,1920,154 Maple Ave,NY,Delanson,Schenectady
1,395000.0,4.0,3.0,2.0,2.0,single_family,30056.0,2987.0,1982,1169 Hidden Valley Trl,NY,Webster,Monroe
2,185000.0,4.0,2.0,1.0,1.0,single_family,7501.0,1863.0,1965,7869 Oneida Trl,NY,Bridgeport,Onondaga
3,440000.0,4.0,3.0,2.0,2.0,single_family,17860.0,1940.0,1965,16 Brookland Farms Rd,NY,Poughkeepsie,Dutchess
4,975700.0,5.0,6.0,3.0,2.0,single_family,25544.0,5660.0,1999,7534 Plum Hollow Cir,NY,Liverpool,Onondaga
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7993,1049000.0,6.0,2.0,1.0,2.0,multi_family,2896.0,1725.0,1920,74-15 88th Ave,NY,Woodhaven,Queens
7994,1495000.0,1.0,1.0,0.0,16.0,coop,10454.0,1725.0,1925,45 5th Ave Apt 17C,NY,New York City,New York
7995,1395000.0,3.0,1.0,0.0,5.0,condos,10454.0,1725.0,1890,705 Carroll St Apt 4R,NY,New York City,Kings
7996,4500000.0,6.0,4.0,0.0,4.0,townhomes,10454.0,1725.0,1958,608 3rd St,NY,New York City,Kings


## Linear Regression model

### Dropping the unwanted columns for the machine learning

In [469]:
forsale_df_ml = forsale_df.copy()
forsale_df_ml.drop(["address","house_type","state","city","county"], axis=1, inplace=True)
forsale_df_ml['address'] = LabelEncoder().fit_transform(forsale_df['address'])
forsale_df_ml['house_type'] = LabelEncoder().fit_transform(forsale_df['house_type'])
forsale_df_ml['state'] = LabelEncoder().fit_transform(forsale_df['state'])
forsale_df_ml['city'] = LabelEncoder().fit_transform(forsale_df['city'])
forsale_df_ml['county'] = LabelEncoder().fit_transform(forsale_df['county'])
repc=forsale_df_ml.pop('price')
forsale_df_ml['price']=repc
forsale_df_ml

,beds,baths,garage,stories,lot_sqft,sqft,year_built,address,house_type,state,city,county,price
0,3.0,2.0,0.0,2.0,12632.0,1190.0,1920,1643,9,0,262,45,139900.0
1,4.0,3.0,2.0,2.0,30056.0,2987.0,1982,737,9,0,1047,27,395000.0
2,4.0,2.0,1.0,1.0,7501.0,1863.0,1965,7096,9,0,120,33,185000.0
3,4.0,3.0,2.0,2.0,17860.0,1940.0,1965,1736,9,0,810,13,440000.0
4,5.0,6.0,3.0,2.0,25544.0,5660.0,1999,6987,9,0,564,33,975700.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7993,6.0,2.0,1.0,2.0,2896.0,1725.0,1920,6955,8,0,1093,40,1049000.0
7994,1.0,1.0,0.0,16.0,10454.0,1725.0,1925,5352,4,0,690,30,1495000.0
7995,3.0,1.0,0.0,5.0,10454.0,1725.0,1890,6823,3,0,690,23,1395000.0
7996,6.0,4.0,0.0,4.0,10454.0,1725.0,1958,6347,10,0,690,23,4500000.0


In [470]:
repc

0        139900.0
1        395000.0
2        185000.0
3        440000.0
4        975700.0
          ...    
7993    1049000.0
7994    1495000.0
7995    1395000.0
7996    4500000.0
7997     570000.0
Name: price, Length: 7998, dtype: float64

In [471]:
# X =forsale_df_ml.drop(['price'],axis=1)
# y =forsale_df_ml['price']
X = forsale_df_ml.iloc[:,:-1]
y = forsale_df_ml.iloc[:,-1]

In [472]:
X

,beds,baths,garage,stories,lot_sqft,sqft,year_built,address,house_type,state,city,county
0,3.0,2.0,0.0,2.0,12632.0,1190.0,1920,1643,9,0,262,45
1,4.0,3.0,2.0,2.0,30056.0,2987.0,1982,737,9,0,1047,27
2,4.0,2.0,1.0,1.0,7501.0,1863.0,1965,7096,9,0,120,33
3,4.0,3.0,2.0,2.0,17860.0,1940.0,1965,1736,9,0,810,13
4,5.0,6.0,3.0,2.0,25544.0,5660.0,1999,6987,9,0,564,33
...,...,...,...,...,...,...,...,...,...,...,...,...
7993,6.0,2.0,1.0,2.0,2896.0,1725.0,1920,6955,8,0,1093,40
7994,1.0,1.0,0.0,16.0,10454.0,1725.0,1925,5352,4,0,690,30
7995,3.0,1.0,0.0,5.0,10454.0,1725.0,1890,6823,3,0,690,23
7996,6.0,4.0,0.0,4.0,10454.0,1725.0,1958,6347,10,0,690,23


In [473]:
y

0        139900.0
1        395000.0
2        185000.0
3        440000.0
4        975700.0
          ...    
7993    1049000.0
7994    1495000.0
7995    1395000.0
7996    4500000.0
7997     570000.0
Name: price, Length: 7998, dtype: float64

## Modeling and scores

In [476]:
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=0 , test_size=0.4)

In [477]:
linear_model = LinearRegression(fit_intercept=False)

linear_model.fit(X_train,y_train)

y_pred = linear_model.predict(X_test)
y_train_pred = linear_model.predict(X_train)
y_pred

array([979608.35348828, 753153.74023949, 147895.11153597, ...,
       739350.450443  , 299452.39261691, 486914.42305565])

In [478]:
print("r2 score with LinearRegression model")
print("Accuracy on training data: " , r2_score(y_train, y_train_pred))
print("Accuracy on test data: ", r2_score(y_test, y_pred))

r2 score with LinearRegression model
Accuracy on training data:  0.27460706305803073
Accuracy on test data:  0.3041054588110935


In [437]:
print (forsale_df.columns.tolist())

['price', 'beds', 'baths', 'garage', 'stories', 'house_type', 'lot_sqft', 'sqft', 'year_built', 'address', 'state', 'city', 'county']


In [438]:
forsale_cp_df_ml=forsale_df.copy()
featurs =forsale_cp_df_ml.drop(["price","county","address","baths","stories"],axis=1)

In [439]:
#create dummies and combine 
combine_dummies = pd.get_dummies(featurs)
result = combine_dummies.values

In [440]:
scaler = StandardScaler()
result = scaler.fit_transform(result)

In [441]:
X= result[:forsale_cp_df_ml.shape[0]]
y=forsale_cp_df_ml['price'].values

In [442]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.4)

In [443]:
lasso_model= Lasso()

lasso_model.fit(X_train, y_train)

y_pred = lasso_model.predict(X_test)
y_train_pred = lasso_model.predict(X_train)
y_pred

array([ 977356.72816037,  458208.68865174,  144435.19796751, ...,
        652968.71915066, 1091314.12822125, 1661384.17778332])

In [444]:
print("r2 score with Lasso model")
print("Accuracy on training data: " , r2_score(y_train, y_train_pred))
print("Accuracy on test data: ", r2_score(y_test, y_pred))

r2 score with Lasso model
Accuracy on training data:  0.6026066045734804
Accuracy on test data:  0.3216388468569674


In [445]:
ols = LinearRegression()
ols.fit(X_train, y_train)
y_ypred = ols.predict(X_test)
y_train_pred = ols.predict(X_train)

In [446]:
print("r2 score with Lasso model")
print("Accuracy on training data: " , r2_score(y_train, y_train_pred))
print("Accuracy on test data: ", r2_score(y_test, y_pred))

r2 score with Lasso model
Accuracy on training data:  0.5093626055050178
Accuracy on test data:  0.3216388468569674


In [447]:
bayesian = BayesianRidge()
bayesian.fit(X_train, y_train)
y_pred = bayesian.predict(X_test)
y_train_pred = bayesian.predict(X_train)

In [448]:
print("r2 score with Lasso model")
print("Accuracy on training data: " , r2_score(y_train, y_train_pred))
print("Accuracy on test data: ", r2_score(y_test, y_pred))

r2 score with Lasso model
Accuracy on training data:  0.5782090476334987
Accuracy on test data:  0.3733451885632364


In [449]:
en = ElasticNet(alpha = 0.01)
en.fit(X_train, y_train)
y_pred = en.predict(X_test)
y_train_pred = en.predict(X_train)

In [450]:
print("r2 score with Lasso model")
print("Accuracy on training data: " , r2_score(y_train, y_train_pred))
print("Accuracy on test data: ", r2_score(y_test, y_pred))

r2 score with Lasso model
Accuracy on training data:  0.6025775690770871
Accuracy on test data:  0.32286209452231396


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'bootstrap': [True],
    'max_depth': [60, 70, 80, 90],
    'max_features': ['auto'],
    'min_samples_leaf': [2, 3, 4],
    'min_samples_split': [2, 4, 6],
    'n_estimators': [100, 144, 300, 600]
    }
rf = RandomForestRegressor()
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
grid_search.fit(X_train,y_train)

Fitting 3 folds for each of 144 candidates, totalling 432 fits


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
#rf_model=grid_search.best_estimator_
rf_model=RandomForestRegressor(bootstrap= True,max_depth=15,max_features=16,min_samples_leaf=2,min_samples_split= 2,n_estimators=144)
rf_model.fit(X_train,y_train)

y_pred = rf_model.predict(X_test)
y_train_pred = rf_model.predict(X_train)

print("r2 score with RendomForestReg model")
print("Accuracy on training data: " , r2_score(y_train, y_train_pred))
print("Accuracy on test data: ", r2_score(y_test, y_pred))